In [ ]:
!pip install patool


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import patoolib
patoolib.extract_archive("/content/drive/MyDrive/dataset/bicep_dumbbell2.zip")

In [ ]:
# Download the model from TF Hub.
import tensorflow_hub as hub
model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
movenet = model.signatures['serving_default']


In [1]:
import os
import glob
import cv2
# import pafy
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt
from PIL import Image

# from moviepy.editor import *
# %matplotlib inline
 
from sklearn.model_selection import train_test_split
 
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from keras.applications.vgg16 import VGG16, preprocess_input


In [2]:
pose_image_size = 192
visibility_threshold = .10

In [ ]:
# Specify the height and width to which each video frame will be resized in our dataset.
# IMAGE_WIDTH = int(1280/4)
# IMAGE_HEIGHT = int(720/4)

IMAGE_WIDTH = 200
IMAGE_HEIGHT = 200




print(IMAGE_WIDTH,IMAGE_HEIGHT)

# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 22

In [4]:
image_paths = glob.glob("/content/bicep_dumbbell2/bicep_dumbbell2/*.avi")

In [ ]:
len(image_paths)

In [6]:
def frames_extraction(video_path):

    # Declare a list to store video frames.
    frames_list = []
    tem_frames = []
    labels_list = []
    video_reader = cv2.VideoCapture(video_path)
    label = int(video_path.split('/')[4].split('_')[1])
    cap = cv2.VideoCapture(video_path)
    count = 0 
    while(cap.isOpened()):
        ret, frame = cap.read()
        count+=1
        try:
          frame = cv2.resize(frame, (IMAGE_WIDTH, IMAGE_HEIGHT))
          frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

          #------Start Cropping images-------------
          image = tf.expand_dims(frame, axis=0)
          image = tf.cast(tf.image.resize_with_pad(image, pose_image_size, pose_image_size), dtype=tf.int32)
          # Run model inference.
          outputs = movenet(image)
          keypoints = outputs['output_0']
          keypoints = keypoints[0][0]
          # get one hand
          right_hand = np.array(np.array(keypoints[10])*IMAGE_WIDTH , dtype=np.int16)
          # Cropping image
          try:
            frame = tf.image.crop_to_bounding_box( np.array(frame), right_hand[0]-40, right_hand[1]-40, 80, 80 )
            # corp_image_arr.append(crop_image)
          except:
            frame = tf.zeros([80, 80], tf.int32)
          # print(np.array(frame).shape)
          #-------------------------------
          normalized_frame = frame / 255
          tem_frames.append(normalized_frame)
          
          if len(tem_frames) == SEQUENCE_LENGTH: 
            frames_list.append(tem_frames)
            tem_frames = []
            labels_list.append(int(label))
        except:
          print(f"last frame of {str(video_path.split('/')[2])} => {count}")
          cap.release()
          cv2.destroyAllWindows()


    cap.release()
    cv2.destroyAllWindows()
    # Return the frames list.
    return np.array(frames_list) , np.array(labels_list)

In [ ]:
# images_list = np.zeros((1,SEQUENCE_LENGTH ,IMAGE_HEIGHT , IMAGE_WIDTH , 3))
images_list = np.zeros((1,SEQUENCE_LENGTH ,80 , 80 , 3))
labels_list = []
for idx ,  image_path in enumerate(image_paths):
  print(idx)
  try:
    images , label = frames_extraction(image_path)
    # print(np.array(images[]).shape)
    images_list = np.concatenate(( images_list, images ), axis=0)
    labels_list = np.concatenate((labels_list , label ) , axis = 0)
  except:
    pass

# remove first element which is used for creatation
images_list = images_list[1:]

In [ ]:
labels_list = [int(x) for x in labels_list]
type(labels_list[0])

In [ ]:
labels_list[0]

In [ ]:
np.array(images_list).shape

In [ ]:
np.array(labels_list).shape

In [ ]:
np.unique(labels_list)

In [ ]:
random_group = np.random.randint(images_list.shape[0])

print(labels_list[random_group])

fig,a =  plt.subplots(3,3 , figsize=(10, 10))
a[0][0].imshow(images_list[random_group][0])
a[0][1].imshow(images_list[random_group][1])
a[0][2].imshow(images_list[random_group][2])
a[1][0].imshow(images_list[random_group][3])
a[1][1].imshow(images_list[random_group][4])
a[1][2].imshow(images_list[random_group][5])
a[2][0].imshow(images_list[random_group][6])
a[2][1].imshow(images_list[random_group][7])
a[2][2].imshow(images_list[random_group][8])

plt.show()

In [ ]:
features_train, features_test, labels_train, labels_test = train_test_split(np.array(images_list), np.array(labels_list), test_size = 0.8, shuffle = True, random_state = 52)

# del(images_list)


In [ ]:
def create_LRCN_model():
    '''
    This function will construct the required LRCN model.
    Returns:
        model: It is the required constructed LRCN model.
    '''



    # We will use a Sequential model for model construction.
    model = Sequential()
    

    # Define the Model Architecture.
    ########################################################################################################################
    
    model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same',activation = 'relu'),
                              input_shape = (SEQUENCE_LENGTH, 80, 80, 3)))
    
    model.add(TimeDistributed(MaxPooling2D((2, 2)))) 
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.25)))

    # model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same',activation = 'relu')))
    # model.add(TimeDistributed(MaxPooling2D((2, 2))))
    # model.add(TimeDistributed(Dropout(0.25)))


    # model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    # model.add(TimeDistributed(MaxPooling2D((2, 2))))
    # model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.25)))
                                      
    model.add(TimeDistributed(Flatten()))
                                      
    model.add(LSTM(300,  activation="tanh" , return_sequences=True))
    # model.add(LSTM(16,  activation="tanh" , return_sequences=True))
    model.add(LSTM(300,  activation="tanh" , return_sequences=True))
    # model.add(LSTM(32,  activation="tanh" , return_sequences=True))
    model.add(LSTM(400,  activation="tanh" , return_sequences=True))
    model.add(LSTM(400,  activation="tanh" ))



    # model.add(tf.keras.layers.Bidirectional( LSTM(200,  activation="tanh" , return_sequences=True)) )
    # model.add(tf.keras.layers.Bidirectional( LSTM(300, activation="tanh")) )



    model.add(Dense( 500, activation = 'tanh') )
    # model.add(Dense( 300, activation = 'tanh') )
    # model.add(Dense( 300, activation = 'tanh') )


                               
    model.add(Dense(len(np.unique(labels_list)), activation = 'softmax'))

    ########################################################################################################################

    # Display the models summary.
    # model.summary()
    
    # Return the constructed LRCN model.
    return model

In [ ]:
# Construct the required LRCN model.
LRCN_model = create_LRCN_model()

#
# Create an Instance of Early Stopping Callback.
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 45, mode = 'min', restore_best_weights = True)


# Compile the model and specify loss function, optimizer and metrics to the model.
LRCN_model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(), 
                       optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), 
                       metrics = ["accuracy"])
# Start training the model.
LRCN_model_training_history = LRCN_model.fit(x = features_train, y = labels_train, epochs = 200, batch_size = 4 , shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback])

In [ ]:
LRCN_model.evaluate(features_test , labels_test)

In [ ]:
for x in range(10):
  selected_test_video = x
  result = LRCN_model.predict(np.expand_dims(features_test[selected_test_video], axis = 0), np.array(labels_test[selected_test_video]))
  print(np.argmax(result))
  print(f"Real label {labels_test[selected_test_video] }")
  print("==========")

In [ ]:
# LRCN_model.save('LRCN_model_22f_v_1.h5')

# **Different Model that can be used**
But it is slow and has similer accuracey as prevois model


In [ ]:
# def create_convlstm_model():

#     model = Sequential()

#     model.add(ConvLSTM2D(filters = 8, kernel_size = (3, 3), activation = 'tanh',data_format = "channels_last",
#                          recurrent_dropout=0.2, return_sequences=True, input_shape = (SEQUENCE_LENGTH,
#                                                                                       80, 80, 3)))
    
#     model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
#     model.add(TimeDistributed(Dropout(0.2)))
    
#     model.add(ConvLSTM2D(filters = 16, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
#                          recurrent_dropout=0.2, return_sequences=True))
    
#     model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
#     model.add(TimeDistributed(Dropout(0.2)))
    
#     model.add(ConvLSTM2D(filters = 28, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
#                          recurrent_dropout=0.2, return_sequences=True))
    
#     model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
#     model.add(TimeDistributed(Dropout(0.2)))
    
#     model.add(ConvLSTM2D(filters = 16, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
#                          recurrent_dropout=0.2, return_sequences=True))
    
#     model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
#     #model.add(TimeDistributed(Dropout(0.2)))
    
#     model.add(Flatten()) 

#     model.add(Dense( 50, activation = 'tanh'))
#     model.add(Dropout(0.3))
#     model.add(Dense( 50, activation = 'tanh'))
#     model.add(Dropout(0.3))    
#     model.add(Dense( 50, activation = 'tanh'))
#     model.add(Dropout(0.3))
    
#     model.add(Dense(len(np.unique(labels_list)), activation = "softmax"))

#     return model


# # Construct the required convlstm model.
# convlstm_model = create_convlstm_model()


# # Create an Instance of Early Stopping Callback
# early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 25, mode = 'min', restore_best_weights = True)

# # Compile the model and specify loss function, optimizer and metrics values to the model
# convlstm_model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(), 
#                        optimizer = 'Adam', 
#                        metrics = ["accuracy"])

# # Start training the model.
# convlstm_model_training_history = convlstm_model.fit(x = features_train, y = labels_train, epochs = 50, batch_size = 4,shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback])
